# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 10:54AM,249513,15,VT80179870,"Virtus Pharmaceuticals, LLC",Released
1,Oct 27 2022 10:54AM,249513,15,VT80179871,"Virtus Pharmaceuticals, LLC",Released
2,Oct 27 2022 10:54AM,249513,15,VT80179872,"Virtus Pharmaceuticals, LLC",Released
3,Oct 27 2022 10:54AM,249513,15,VT80179924,"Virtus Pharmaceuticals, LLC",Released
4,Oct 27 2022 10:53AM,249512,15,8695720,"Brookfield Pharmaceuticals, LLC",Released
5,Oct 27 2022 10:53AM,249511,15,ALIA520265-01,"Alliance Pharma, Inc.",Released
6,Oct 27 2022 10:53AM,249511,15,ALIL372059-01,"Alliance Pharma, Inc.",Executing
7,Oct 27 2022 10:47AM,249510,10,8699536,"ClearSpec, LLC",Released
8,Oct 27 2022 10:45AM,249509,19,ACG-2102491697,ACG North America LLC,Released
9,Oct 27 2022 10:37AM,249472,15,30011750,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249510,Released,1
18,249511,Executing,1
19,249511,Released,1
20,249512,Released,1
21,249513,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249509,NaN,NaN,1.0
249510,NaN,NaN,1.0
249511,NaN,1.0,1.0
249512,NaN,NaN,1.0
249513,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0.0,0.0,1.0
248599,0.0,0.0,1.0
249451,0.0,1.0,0.0
249472,0.0,4.0,37.0
249479,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0,0,1
248599,0,0,1
249451,0,1,0
249472,0,4,37
249479,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,0,0,1
1,248599,0,0,1
2,249451,0,1,0
3,249472,0,4,37
4,249479,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,,,1
1,248599,,,1
2,249451,,1,
3,249472,,4,37
4,249479,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC"
4,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC"
5,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc."
7,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC"
8,Oct 27 2022 10:45AM,249509,19,ACG North America LLC
9,Oct 27 2022 10:37AM,249472,15,"Alliance Pharma, Inc."
50,Oct 27 2022 10:04AM,249502,21,"NBTY Global, Inc."
51,Oct 27 2022 9:47AM,249499,10,Bio-PRF
62,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc."
66,Oct 27 2022 9:08AM,249492,12,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC",,,4
1,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC",,,1
2,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc.",,1,1
3,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC",,,1
4,Oct 27 2022 10:45AM,249509,19,ACG North America LLC,,,1
5,Oct 27 2022 10:37AM,249472,15,"Alliance Pharma, Inc.",,4,37
6,Oct 27 2022 10:04AM,249502,21,"NBTY Global, Inc.",,,1
7,Oct 27 2022 9:47AM,249499,10,Bio-PRF,,7,4
8,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",,3,1
9,Oct 27 2022 9:08AM,249492,12,SugarBear,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC",4,,
1,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC",1,,
2,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc.",1,1,
3,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC",1,,
4,Oct 27 2022 10:45AM,249509,19,ACG North America LLC,1,,
5,Oct 27 2022 10:37AM,249472,15,"Alliance Pharma, Inc.",37,4,
6,Oct 27 2022 10:04AM,249502,21,"NBTY Global, Inc.",1,,
7,Oct 27 2022 9:47AM,249499,10,Bio-PRF,4,7,
8,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",1,3,
9,Oct 27 2022 9:08AM,249492,12,SugarBear,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC",4,,
1,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC",1,,
2,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc.",1,1,
3,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC",1,,
4,Oct 27 2022 10:45AM,249509,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC",4.0,NaN,NaN
1,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
2,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc.",1.0,1.0,NaN
3,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Oct 27 2022 10:45AM,249509,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 10:54AM,249513,15,"Virtus Pharmaceuticals, LLC",4.0,0.0,0.0
1,Oct 27 2022 10:53AM,249512,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
2,Oct 27 2022 10:53AM,249511,15,"Alliance Pharma, Inc.",1.0,1.0,0.0
3,Oct 27 2022 10:47AM,249510,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Oct 27 2022 10:45AM,249509,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,997992,7.0,18.0,5.0
12,249492,1.0,0.0,0.0
15,998008,43.0,5.0,0.0
16,249479,0.0,1.0,0.0
19,249509,1.0,0.0,0.0
20,249485,0.0,1.0,0.0
21,995901,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,997992,7.0,18.0,5.0
1,12,249492,1.0,0.0,0.0
2,15,998008,43.0,5.0,0.0
3,16,249479,0.0,1.0,0.0
4,19,249509,1.0,0.0,0.0
5,20,249485,0.0,1.0,0.0
6,21,995901,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,18.0,5.0
1,12,1.0,0.0,0.0
2,15,43.0,5.0,0.0
3,16,0.0,1.0,0.0
4,19,1.0,0.0,0.0
5,20,0.0,1.0,0.0
6,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,1.0
2,15,Released,43.0
3,16,Released,0.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,5.0,1.0,0.0,1.0,1.0
Released,7.0,1.0,43.0,0.0,1.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,5.0,1.0,0.0,1.0,1.0
2,Released,7.0,1.0,43.0,0.0,1.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,5.0,1.0,0.0,1.0,1.0
2,Released,7.0,1.0,43.0,0.0,1.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()